In [1]:
from handcalcs_adapter.units import *
import math
from math import (
    sqrt, 
    sin, cos, tan, radians)

import handcalcs
from handcalcs import render
handcalcs.set_option("display_precision", 2)
handcalcs.set_option("param_columns", 1)

In [ ]:
from handcalcs_adapter.section import section, Section
sec = section(Section.C200x75x20x2_8)

In [3]:
%%render input

## 1. Input

## Purlin geometry
L = (8.0 * m) # Purlin span
S = (1.2 * m) # Purlin spacing
theta = (radians( 15.0 )) # Purlin roll angle from strong axis

## Sag-rod
N_sagrod = 1

## Load
## Load design
q_DL = (0.3 * kN/(m**2)) # dead load
q_LL = (0.24 * kN/(m**2)) # live load
q_WL = (0.4 * kN/(m**2)) # wind load
# Catatan:
# - Live load
#     bangunan pabrik : 0.24 kN/m2

## Material
E = (200_000 * MPa) # Young's modulus
F_y = (240 * MPa) # Yield strength


## 1. Input

Purlin geometry

$
\hspace{2em}\begin{aligned}
L &= 8.00\ \mathrm{m} \; \;\textrm{(Purlin span)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
S &= 1.20\ \mathrm{m} \; \;\textrm{(Purlin spacing)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
\theta &= 0.26 \; \;\textrm{(Purlin roll angle from strong axis)}
\end{aligned}
$

Sag-rod

$
\hspace{2em}\begin{aligned}
N_{sagrod} &= 1 \;
\end{aligned}
$

Load

Load design

$
\hspace{2em}\begin{aligned}
q_{DL} &= 300.00\ \mathrm{Pa} \; \;\textrm{(dead load)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
q_{LL} &= 240.00\ \mathrm{Pa} \; \;\textrm{(live load)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
q_{WL} &= 400.00\ \mathrm{Pa} \; \;\textrm{(wind load)}
\end{aligned}
$

Material

$
\hspace{2em}\begin{aligned}
E &= 200.00\ \mathrm{GPa} \; \;\textrm{(Young's modulus)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
F_{y} &= 240.00\ \mathrm{MPa} \; \;\textrm{(Yield strength)}
\end{aligned}
$

In [4]:
%%render input

## 2. Selec purlin
section_name = (sec.name)
w_self = (sec.w) # self weight
I_x = (sec.Ix) # strong-axis moment of inertia
I_y = (sec.Iy) # weak-axis moment of inertia
Z_x = (sec.Zx) # strong-axis modulus of section
Z_y = (sec.Zy) # weak-axis modulus of section

## 2. Selec purlin

$
\hspace{2em}\begin{aligned}
\mathrm{section}_{name} &= \mathrm{C200x75x20x2.8} \;
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
w_{self} &= 80.15\ \mathrm{N/m} \; \;\textrm{(self weight)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
I_{x} &= 6360000.00\ \mathrm{mm}^{4} \; \;\textrm{(strong-axis moment of inertia)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
I_{y} &= 750000.00\ \mathrm{mm}^{4} \; \;\textrm{(weak-axis moment of inertia)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
Z_{x} &= 63600.00\ \mathrm{mm}^{3} \; \;\textrm{(strong-axis modulus of section)}
\end{aligned}
$

$
\hspace{2em}\begin{aligned}
Z_{y} &= 14200.00\ \mathrm{mm}^{3} \; \;\textrm{(weak-axis modulus of section)}
\end{aligned}
$

In [5]:
%%render report

## 3. Calculation

## 3.1 Check for deflection

## Design load for deflection
w_DL = q_DL * S
w_LL = q_LL * S

## Design load for deflection
w = w_DL + w_LL + w_self

## Load resolution for each axis
w_x = w * cos(theta)
w_y = w * sin(theta)

## Deflection allowable
Delta_allowable = L / 240

## Calculate deflection
## ---
## On X axis
L_x = L
Delta_x = (5 * w_x * L_x**4) / (384 * E * I_x)

## On Y axis
L_y = L / (N_sagrod + 1)
Delta_y = (5 * w_y   * L_y**4) / (384 * E * I_y)

## Total deflection
Delta_total = (Delta_x**2 + Delta_y**2) ** 0.5

if Delta_total <= Delta_allowable : Delta = Delta_allowable # OK
elif Delta_total > Delta_allowable : Delta = Delta_allowable # NOT.OK

## 3.1 Check for flexural strength

## Design load for flexural strength
w_DL = q_DL * S
w_LL = q_LL * S
w_WL = q_WL * S

## Design load for deflection
w1 = 1.2*(w_DL + w_self) + 1.6*w_LL + 0.5*w_WL
w2 = 1.2*(w_DL + w_self) + 1*w_LL + 1*w_WL
w = max(w1, w2)

## Load resolution for each eaxis
w_x = w * cos(theta)
w_y = w * sin(theta)

## Calculate moment ultimate
## ---
## On X axis
L_x = L
M_UX = 1/8 * w_x * (L_x ** 2)

## On Y axis
L_y = L / (N_sagrod + 1)
M_UY = 1/8 * w_y * (L_y ** 2)

## Calculate flexural strength
## ---
## On X axis
M_nX = 0.9 * F_y * Z_x

## On Y axis
M_nY = 0.9 * F_y * Z_y

if M_UX <= M_nX : M_nX = M_nX # OK
elif M_UX > M_nX : M_nX = M_nX # NOT.OK 
if M_UY <= M_nY : M_nY = M_nY # OK
elif M_UY > M_nY : M_nY = M_nY # NOT.OK 


## 3. Calculation

### 3.1 Check for deflection

Design load for deflection

$$
\begin{aligned}
w_{DL} &= q_{DL} \cdot S \\&= 300.00\ \mathrm{Pa} \cdot 1.20\ \mathrm{m} \\&= 360.00\ \mathrm{N/m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
w_{LL} &= q_{LL} \cdot S \\&= 240.00\ \mathrm{Pa} \cdot 1.20\ \mathrm{m} \\&= 288.00\ \mathrm{N/m}  \\[10pt]
\end{aligned}
$$

Design load for deflection

$$
\begin{aligned}
w &= w_{DL} + w_{LL} + w_{self} \\&= 360.00\ \mathrm{N/m} + 288.00\ \mathrm{N/m} + 80.15\ \mathrm{N/m} \\&= 1.30\ \mathrm{kN/m}  \\[10pt]
\end{aligned}
$$

Load resolution for each axis

$$
\begin{aligned}
w_{x} &= w \cdot \cos \left( \theta \right) \\&= 1.30\ \mathrm{kN/m} \cdot \cos \left( 0.26 \right) \\&= 1.25\ \mathrm{kN/m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
w_{y} &= w \cdot \sin \left( \theta \right) \\&= 1.30\ \mathrm{kN/m} \cdot \sin \left( 0.26 \right) \\&= 335.48\ \mathrm{N/m}  \\[10pt]
\end{aligned}
$$

Deflection allowable

$$
\begin{aligned}
\Delta_{allowable} &= \frac{ L }{ 240 } \\&= \frac{ 8.00\ \mathrm{m} }{ 240 } \\&= 33.33\ \mathrm{mm}  \\[10pt]
\end{aligned}
$$

Calculate deflection

---

On X axis

$$
\begin{aligned}
L_{x} &= 8.00\ \mathrm{m} \; 
\end{aligned}
$$

$$
\begin{aligned}
\Delta_{x} &= \frac{ 5 \cdot w_{x} \cdot \left( L_{x} \right) ^{ 4 } }{ 384 \cdot E \cdot I_{x} } \\&= \frac{ 5 \cdot 1.25\ \mathrm{kN/m} \cdot \left( 8.00\ \mathrm{m} \right) ^{ 4 } }{ 384 \cdot 200.00\ \mathrm{GPa} \cdot 6360000.00\ \mathrm{mm}^{4} } \\&= 29.49\ \mathrm{mm}  \\[10pt]
\end{aligned}
$$

On Y axis

$$
\begin{aligned}
L_{y} &= \frac{ L }{ N_{sagrod} + 1 } \\&= \frac{ 8.00\ \mathrm{m} }{ 1 + 1 } \\&= 4.00\ \mathrm{m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
\Delta_{y} &= \frac{ 5 \cdot w_{y} \cdot \left( L_{y} \right) ^{ 4 } }{ 384 \cdot E \cdot I_{y} } \\&= \frac{ 5 \cdot 335.48\ \mathrm{N/m} \cdot \left( 4.00\ \mathrm{m} \right) ^{ 4 } }{ 384 \cdot 200.00\ \mathrm{GPa} \cdot 750000.00\ \mathrm{mm}^{4} } \\&= 4.19\ \mathrm{mm}  \\[10pt]
\end{aligned}
$$

Total deflection

$$
\begin{aligned}
\Delta_{total} &= \left( \left( \Delta_{x} \right) ^{ 2 } + \left( \Delta_{y} \right) ^{ 2 } \right) ^{ 0.5 } \\&= \left( \left( 29.49\ \mathrm{mm} \right) ^{ 2 } + \left( 4.19\ \mathrm{mm} \right) ^{ 2 } \right) ^{ 0.5 } \\&= 29.79\ \mathrm{mm}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
&\text{Since, } \Delta_{total} \leq \Delta_{allowable} \rightarrow \left( 29.79\ \mathrm{mm} \leq 33.33\ \mathrm{mm} \right) : \; \;\textrm{(OK)} \\[10pt]
\Delta &= 33.33\ \mathrm{mm} \; 
\end{aligned}
$$

### 3.1 Check for flexural strength

Design load for flexural strength

$$
\begin{aligned}
w_{DL} &= q_{DL} \cdot S \\&= 300.00\ \mathrm{Pa} \cdot 1.20\ \mathrm{m} \\&= 360.00\ \mathrm{N/m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
w_{LL} &= q_{LL} \cdot S \\&= 240.00\ \mathrm{Pa} \cdot 1.20\ \mathrm{m} \\&= 288.00\ \mathrm{N/m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
w_{WL} &= q_{WL} \cdot S \\&= 400.00\ \mathrm{Pa} \cdot 1.20\ \mathrm{m} \\&= 480.00\ \mathrm{N/m}  \\[10pt]
\end{aligned}
$$

Design load for deflection

$$
\begin{aligned}
\mathrm{w1} &= 1.2 \cdot \left( w_{DL} + w_{self} \right) + 1.6 \cdot w_{LL} + 0.5 \cdot w_{WL} \\&= 1.2 \cdot \left( 360.00\ \mathrm{N/m} + 80.15\ \mathrm{N/m} \right) + 1.6 \cdot 288.00\ \mathrm{N/m} + 0.5 \cdot 480.00\ \mathrm{N/m} \\&= 1.23\ \mathrm{kN/m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
\mathrm{w2} &= 1.2 \cdot \left( w_{DL} + w_{self} \right) + 1 \cdot w_{LL} + 1 \cdot w_{WL} \\&= 1.2 \cdot \left( 360.00\ \mathrm{N/m} + 80.15\ \mathrm{N/m} \right) + 1 \cdot 288.00\ \mathrm{N/m} + 1 \cdot 480.00\ \mathrm{N/m} \\&= 1.30\ \mathrm{kN/m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
w &= \operatorname{max} \left( \mathrm{w1} ,\  \mathrm{w2} \right) \\&= \operatorname{max} \left( 1.23\ \mathrm{kN/m} ,\  1.30\ \mathrm{kN/m} \right) \\&= 1.30\ \mathrm{kN/m}  \\[10pt]
\end{aligned}
$$

Load resolution for each eaxis

$$
\begin{aligned}
w_{x} &= w \cdot \cos \left( \theta \right) \\&= 1.30\ \mathrm{kN/m} \cdot \cos \left( 0.26 \right) \\&= 1.25\ \mathrm{kN/m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
w_{y} &= w \cdot \sin \left( \theta \right) \\&= 1.30\ \mathrm{kN/m} \cdot \sin \left( 0.26 \right) \\&= 335.48\ \mathrm{N/m}  \\[10pt]
\end{aligned}
$$

Calculate moment ultimate

---

On X axis

$$
\begin{aligned}
L_{x} &= 8.00\ \mathrm{m} \; 
\end{aligned}
$$

$$
\begin{aligned}
M_{UX} &= \frac{ 1 }{ 8 } \cdot w_{x} \cdot \left( L_{x} \right) ^{ 2 } \\&= \frac{ 1 }{ 8 } \cdot 1.25\ \mathrm{kN/m} \cdot \left( 8.00\ \mathrm{m} \right) ^{ 2 } \\&= 10.02\ \mathrm{kN} \cdot \mathrm{m}  \\[10pt]
\end{aligned}
$$

On Y axis

$$
\begin{aligned}
L_{y} &= \frac{ L }{ N_{sagrod} + 1 } \\&= \frac{ 8.00\ \mathrm{m} }{ 1 + 1 } \\&= 4.00\ \mathrm{m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
M_{UY} &= \frac{ 1 }{ 8 } \cdot w_{y} \cdot \left( L_{y} \right) ^{ 2 } \\&= \frac{ 1 }{ 8 } \cdot 335.48\ \mathrm{N/m} \cdot \left( 4.00\ \mathrm{m} \right) ^{ 2 } \\&= 670.95\ \mathrm{N} \cdot \mathrm{m}  \\[10pt]
\end{aligned}
$$

Calculate flexural strength

---

On X axis

$$
\begin{aligned}
M_{nX} &= 0.9 \cdot F_{y} \cdot Z_{x} \\&= 0.9 \cdot 240.00\ \mathrm{MPa} \cdot 63600.00\ \mathrm{mm}^{3} \\&= 13.74\ \mathrm{kN} \cdot \mathrm{m}  \\[10pt]
\end{aligned}
$$

On Y axis

$$
\begin{aligned}
M_{nY} &= 0.9 \cdot F_{y} \cdot Z_{y} \\&= 0.9 \cdot 240.00\ \mathrm{MPa} \cdot 14200.00\ \mathrm{mm}^{3} \\&= 3.07\ \mathrm{kN} \cdot \mathrm{m}  \\[10pt]
\end{aligned}
$$

$$
\begin{aligned}
&\text{Since, } M_{UX} \leq M_{nX} \rightarrow \left( 10.02\ \mathrm{kN} \cdot \mathrm{m} \leq 13.74\ \mathrm{kN} \cdot \mathrm{m} \right) : \; \;\textrm{(OK)} \\[10pt]
M_{nX} &= 13.74\ \mathrm{kN} \cdot \mathrm{m} \; 
\end{aligned}
$$

$$
\begin{aligned}
&\text{Since, } M_{UY} \leq M_{nY} \rightarrow \left( 670.95\ \mathrm{N} \cdot \mathrm{m} \leq 3.07\ \mathrm{kN} \cdot \mathrm{m} \right) : \; \;\textrm{(OK)} \\[10pt]
M_{nY} &= 3.07\ \mathrm{kN} \cdot \mathrm{m} \; 
\end{aligned}
$$